# Flow vs obs flow & speed vs obs speed

Side-by-side comparison of what seems to be related columns.

## What we learned
* **use `flow`, `speed` columns** and ignore `obs_flow`, `obs_speed`
* `flow` is always >= `obs_flow`
* Most of the time, `flow = obs_flow`
* Why would imputation be more than what's observed? Is this only true when `obs_flow == 0`?
   * It does appear like when `obs_flow has mean = 0`, `flow` will be higher and hold non-zeroes.
* We'll just use `flow` for now, and use imputed values always?
* Most cases are `speed < obs_speed` (so imputed tends to be less than what detector says).
* Looking at descriptives, these are occurring where observed speed is perceived to be too high so that these are adjusted down, though means don't differ by too much for the subset where `speed < obs_speed`.

In [1]:
import pandas as pd

from utils import PROCESSED_GCS

In [2]:
def metric_vs_observed(metric: str, filtering: tuple) -> pd.DataFrame:
    PREFIX = "station_weekday_hour"

    metric_df = pd.read_parquet(
        f"{PROCESSED_GCS}{PREFIX}_{metric}.parquet",
        filters = filtering
    )

    obs_metric_df = pd.read_parquet(
        f"{PROCESSED_GCS}{PREFIX}_obs_{metric}.parquet",
        filters = filtering
    )
    
    merge_cols = ["station_uuid", "year", "month", "weekday", "hour"]

    df = pd.merge(
        metric_df,
        obs_metric_df,
        on = merge_cols,
        how = "outer"
    )
    
    return df

In [3]:
flow_df = metric_vs_observed("flow", filtering = None)
speed_df = metric_vs_observed("speed", filtering = None)

In [4]:
def lane_comparisons(df: pd.DataFrame, lane_number: int, metric: str):

    N_ROWS = len(df)
    
    def rounded(numerator, denominator):
        return round(numerator / denominator, 3)
    
    col = f"lane_{lane_number}_{metric}"
    obs_col = f"lane_{lane_number}_obs_{metric}"
    
    display(df[[col, obs_col]].describe())
    
    N_EQUAL = df[df[col] == df[obs_col]].shape[0]
    N_MORE = df[df[col] > df[obs_col]].shape[0]
    N_LESS = df[df[col] < df[obs_col]].shape[0]
    
    
    print(f"# rows: {N_ROWS}")
    print(f"equal: {N_EQUAL}, imputed > obs: {N_MORE}, imputed < obs: {N_LESS}")
    print(f"% equal {rounded(N_EQUAL, N_ROWS)}")
    print(f"greater: {rounded(N_MORE, N_ROWS)}, less: {rounded(N_LESS, N_ROWS)}")
    
    if metric == "speed":
        print("**** values when imputed < obs ****")

        less_df = df.loc[df[col] < df[obs_col]]

        display(less_df[[col, obs_col]].describe())
    
    print("****values when imputed > obs *****")
    
    more_df = df.loc[df[col] > df[obs_col]]
    
    more_df = more_df.assign(
        obs_col_zero = more_df.apply(
            lambda x: True if x[obs_col]==0 
            else False, axis=1)
    )
    
    print(more_df.obs_col_zero.value_counts())


## Flow vs observed flow diagnostics

In [5]:
for i in range(1, 9):
    lane_comparisons(flow_df, i, "flow")

,lane_1_flow,lane_1_obs_flow
count,5490744.0,5524535.0
mean,761.029763,722.35622
std,564.397175,562.738313
min,0.0,0.0
25%,246.4,207.4
50%,699.0,645.75
75%,1210.0,1164.333333
max,5198.0,5198.0


# rows: 5524535
equal: 4826068, imputed > obs: 664676, imputed < obs: 0
% equal 0.874
greater: 0.12, less: 0.0
****values when imputed > obs *****
False    566253
True      98423
Name: obs_col_zero, dtype: int64


,lane_2_flow,lane_2_obs_flow
count,3668484.0,5524535.0
mean,912.07882,577.289599
std,544.438406,606.562733
min,0.0,0.0
25%,400.4,0.0
50%,965.0,347.0
75%,1354.5,1131.333333
max,4737.0,4737.0


# rows: 5524535
equal: 3219391, imputed > obs: 449093, imputed < obs: 0
% equal 0.583
greater: 0.081, less: 0.0
****values when imputed > obs *****
False    372805
True      76288
Name: obs_col_zero, dtype: int64


,lane_3_flow,lane_3_obs_flow
count,3346786.0,5524535.0
mean,829.965098,478.92817
std,480.593977,540.801227
min,0.0,0.0
25%,386.75,0.0
50%,861.25,233.0
75%,1217.0,952.75
max,4496.0,4496.0


...

# rows: 5524535
equal: 137721, imputed > obs: 20006, imputed < obs: 0
% equal 0.025
greater: 0.004, less: 0.0
****values when imputed > obs *****
False    17530
True      2476
Name: obs_col_zero, dtype: int64


,lane_7_flow,lane_7_obs_flow
count,4202.0,5524535.0
mean,481.056592,0.350383
std,215.572484,14.063007
min,18.0,0.0
25%,320.6875,0.0
50%,549.45,0.0
75%,643.25,0.0
max,1051.666667,1027.5


# rows: 5524535
equal: 3624, imputed > obs: 578, imputed < obs: 0
% equal 0.001
greater: 0.0, less: 0.0
****values when imputed > obs *****
False    543
True      35
Name: obs_col_zero, dtype: int64


,lane_8_flow,lane_8_obs_flow
count,0.0,5524535.0
mean,<NA>,0.0
std,<NA>,0.0
min,<NA>,0.0
25%,<NA>,0.0
50%,<NA>,0.0
75%,<NA>,0.0
max,<NA>,0.0


# rows: 5524535
equal: 0, imputed > obs: 0, imputed < obs: 0
% equal 0.0
greater: 0.0, less: 0.0
****values when imputed > obs *****
Series([], Name: obs_col_zero, dtype: int64)


## Speed vs observed speed diagnostics

In [6]:
for i in range(1, 9):
    lane_comparisons(speed_df, i, "speed")

,lane_1_speed,lane_1_obs_speed
count,5490744.0,5410636.0
mean,65.605164,65.664043
std,10.380544,10.444326
min,3.0,3.0
25%,63.625,63.675
50%,66.075,66.233333
75%,73.1,73.3
max,90.5,90.5


# rows: 5524535
equal: 4874919, imputed > obs: 262289, imputed < obs: 273428
% equal 0.882
greater: 0.047, less: 0.049
**** values when imputed < obs ****


,lane_1_speed,lane_1_obs_speed
count,273428.0,273428.0
mean,66.618133,67.894924
std,9.446861,9.228313
min,3.45,4.1
25%,64.4,64.9
50%,68.75,70.9
75%,73.133333,74.525
max,84.725,88.4


****values when imputed > obs *****
False    262289
Name: obs_col_zero, dtype: int64


,lane_2_speed,lane_2_obs_speed
count,3668484.0,3606628.0
mean,64.512993,64.604668
std,9.864657,9.928083
min,3.0,3.0
25%,62.85,63.0
50%,67.45,67.65
75%,70.45,70.525
max,89.675,89.675


...

****values when imputed > obs *****
False    208
Name: obs_col_zero, dtype: int64


,lane_8_speed,lane_8_obs_speed
count,0.0,0.0
mean,<NA>,<NA>
std,<NA>,<NA>
min,<NA>,<NA>
25%,<NA>,<NA>
50%,<NA>,<NA>
75%,<NA>,<NA>
max,<NA>,<NA>


# rows: 5524535
equal: 0, imputed > obs: 0, imputed < obs: 0
% equal 0.0
greater: 0.0, less: 0.0
**** values when imputed < obs ****


,lane_8_speed,lane_8_obs_speed
count,0.0,0.0
mean,<NA>,<NA>
std,<NA>,<NA>
min,<NA>,<NA>
25%,<NA>,<NA>
50%,<NA>,<NA>
75%,<NA>,<NA>
max,<NA>,<NA>


****values when imputed > obs *****
Series([], Name: obs_col_zero, dtype: int64)
